In [1]:
import json
import os
import shutil
from pathlib import Path

In [2]:
# 1. Load the settings
settings_file = 'windows-reset-settings.json'
with open(settings_file, 'r') as f:
    settings = json.load(f)

backup_root = Path(settings['backup-dir'])
apps = settings['applications']

print(f"Starting backup to: {backup_root}")

Starting backup to: H:\windows-backup


In [3]:
# 2. Handle the 'regs' folder
source_regs = Path('regs')
target_regs = backup_root / 'regs'

if source_regs.exists():
    if target_regs.exists():
        print("Cleaning existing 'regs' folder in backup directory...")
        shutil.rmtree(target_regs)
    
    shutil.copytree(source_regs, target_regs)
    print("Successfully backed up 'regs' folder.")
else:
    print("Warning: Local 'regs' folder not found. Skipping.")

Cleaning existing 'regs' folder in backup directory...
Successfully backed up 'regs' folder.


In [4]:
def sanitize_name(path_str):
    """Converts a path like %appdata%\\discord into appdata_discord."""
    return path_str.replace('%', '').replace('\\', '_').replace(':', '').replace('.', '_').strip('_')

def force_copy_with_tracking(src, dst):
    """
    Recursively copies files. 
    Returns (success_count, fail_count)
    """
    s_count = 0
    f_count = 0
    
    src = Path(src)
    dst = Path(dst)

    if src.is_dir():
        try:
            dst.mkdir(parents=True, exist_ok=True)
        except Exception as e:
            print(f"    [!] Critical: Could not create directory {dst.name}: {e}")
            return 0, 1
            
        for item in os.listdir(src):
            s, f = force_copy_with_tracking(src / item, dst / item)
            s_count += s
            f_count += f
    else:
        try:
            shutil.copy2(src, dst)
            s_count += 1
        except Exception as e:
            print(f"    [!] Failed to copy file: {src.name} (Reason: {e})")
            f_count += 1
            
    return s_count, f_count

In [ ]:
# 3. Process Applications
for app_name, paths in apps.items():
    START_LABEL = "[+] Backing up:"
    FINISH_LABEL = "[»] Finished:  "
    
    print(f"\nProcessing application: {app_name}")
    
    app_folder = backup_root / app_name
    (app_folder / 'screenshots').mkdir(parents=True, exist_ok=True)
    backups_folder = app_folder / 'backups'
    backups_folder.mkdir(parents=True, exist_ok=True)
    
    for raw_path in paths:
        expanded_path_str = os.path.expandvars(raw_path)
        source_path = Path(expanded_path_str)
        
        if not source_path.exists():
            print(f"  [!] Path not found: {expanded_path_str}")
            continue

        folder_id = sanitize_name(raw_path)
        unique_item_dir = backups_folder / folder_id
        
        # Atomic Update Logic
        temp_dir = None
        if unique_item_dir.exists():
            temp_dir = unique_item_dir.with_name(f"{folder_id}_old_tmp")
            if temp_dir.exists(): shutil.rmtree(temp_dir)
            unique_item_dir.rename(temp_dir)
            
        unique_item_dir.mkdir(parents=True, exist_ok=True)
        
        # Create metadata file
        with open(unique_item_dir / f"{folder_id}.txt", 'w') as f:
            f.write(str(source_path))
            
        destination = unique_item_dir / source_path.name
        
        # Aligned Output
        print(f"  {START_LABEL} {folder_id}")
        
        try:
            successes, failures = force_copy_with_tracking(source_path, destination)
            print(f"  {FINISH_LABEL} {folder_id}; {successes} files copied, {failures} skipped")
            
            if temp_dir and temp_dir.exists():
                shutil.rmtree(temp_dir)
                
        except Exception as e:
            print(f"  [!] CRITICAL: {e}")
            if temp_dir and temp_dir.exists():
                print(f"  [!] Rollback: Restoring previous backup for {folder_id}")
                if unique_item_dir.exists(): shutil.rmtree(unique_item_dir)
                temp_dir.rename(unique_item_dir)


Processing application: discord
  [+] Backing up: appdata_discord
    [!] Failed to copy file: data_0 (Reason: [Errno 13] Permission denied: 'C:\\Users\\Josh\\AppData\\Roaming\\discord\\Cache\\Cache_Data\\data_0')
    [!] Failed to copy file: data_1 (Reason: [Errno 13] Permission denied: 'C:\\Users\\Josh\\AppData\\Roaming\\discord\\Cache\\Cache_Data\\data_1')
    [!] Failed to copy file: data_2 (Reason: [Errno 13] Permission denied: 'C:\\Users\\Josh\\AppData\\Roaming\\discord\\Cache\\Cache_Data\\data_2')
    [!] Failed to copy file: data_3 (Reason: [Errno 13] Permission denied: 'C:\\Users\\Josh\\AppData\\Roaming\\discord\\Cache\\Cache_Data\\data_3')
